In [13]:
import chess.engine
import chess 
import matplotlib.pyplot as plt
import chess.pgn 
from stockfish import Stockfish
import pandas as pd
import time
import numpy as np
import json

# Executing the Maia Move Match

In this file we execute the maia move match functions on each of the games for each of the different Maia Engines. We then save our results under the curated folder in the data section. Please note that each move match call for the 5000 games takes approximately 70 minutes so do not run again!!!

In [2]:
def change_config(model):
    """
    Takes a string of the requested model, then modifies the config file
    so that maia uses said model.
    """
    with open("lc0_windows\lc0.config", "w") as file:
        file.write(model)

    with open('lc0_windows\lc0.config', 'r') as file:
        updated_content = file.read()
    print(updated_content)


def get_move(board, engine, engine_str):
    """
    Takes the board engine and engine type and returns the best move based on that.
    """
    if engine_str == "maia":
        maia_move = str(engine.play(board, chess.engine.Limit(time=0.0)).move)
        return maia_move
    elif engine_str == "stockfish":
        engine.set_fen_position(board.fen())
        return engine.get_best_move()

def apply_engine_v2(row, engine, engine_str):
    """
    Takes in a row, and outputs the move match for a given maia engine.
    """
    moves = row["Moves"]
    if len(moves) < 7:
        return None
    mm_w = []
    mm_b = []
    board = chess.Board()
    
    # exclude the first 6 moves in our analysis
    for j in range(6):
        board.push_san(moves[j])
        
    for i in range(6, len(moves)):
    
        if i%2 == 0:
            mm_w.append(get_move(board, engine, engine_str) == moves[i])

        else:
        #if i%2 == 1:
            mm_b.append(get_move(board, engine, engine_str) == moves[i])

        board.push_san(moves[i])

    # Calculate the proportion of True values
    proportion_true_w = sum(mm_w) / len(mm_w)
    proportion_true_b = sum(mm_b) / len(mm_b)    

    w_out = {
        "elo": int(row["BlackElo"]),
        "mm": proportion_true_b
    }
    w_out = {
        "elo": int(row["WhiteElo"]),
        "mm": proportion_true_w
    }
    
    return [w_out, w_out]

def execute_maia_mm_v2(model, games):
    """
    Executes the move matching algorithms and returns a list of dictionaries {elo: , mm: }
    """
    change_config(model)
    maia = chess.engine.SimpleEngine.popen_uci(["lc0_windows\lc0.exe"])
    mm = []

    for game in games:
        try:
            move_matches = apply_engine_v2(game, maia, "maia")
            mm.append(move_matches[0])
            mm.append(move_matches[1])
        except:
            continue
    maia.quit()

    return mm


def execute_stockfish_mm(games, depth):
    """
    Executes the move matching algorithms and returns a list of dictionaries {elo: , mm: }
    """
    mm = []
    stockfish=Stockfish("../stockfish/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_depth(depth)

    for game in games:
        try:
            move_matches = apply_engine_v2(game, stockfish, "stockfish")
            mm.append(move_matches[0])
            mm.append(move_matches[1])
        except:
            continue

    return mm


In [69]:
def closest_100(x):
    if x < 1100:
        return 1100
    elif x > 1900:
        return 1900
    else:
        return round(x / 100) * 100


def maia_cpl(row, stockfish):
    """ returns the cpl of maia engine for both black and white"""

    cpl_w = []
    cpl_b = []

    # defining closest maia model based on the white elo
    model_w = f"--weights=lc0_windows\models\maia-{closest_100(int(row['WhiteElo']))}.pb.gz"
    change_config(model_w)
    maia_w = chess.engine.SimpleEngine.popen_uci(["lc0_windows\lc0.exe"])

    # defining closest maia model based on the black elo
    model_b = f"--weights=lc0_windows\models\maia-{closest_100(int(row['BlackElo']))}.pb.gz"
    change_config(model_b)
    maia_b = chess.engine.SimpleEngine.popen_uci(["lc0_windows\lc0.exe"])

    # making sure there are more than 6 moves
    moves = row["Moves"]
    if len(moves) < 7:
        return None
    
    board = chess.Board()
    
    # exclude the first 6 moves in our analysis
    for j in range(6):
        board.push_san(moves[j])
        
    for i in range(6, len(moves)):
    
        if i%2 == 0:
            try:
                maia_move = get_move(board, maia_w, "maia")
                cpl_w.append(get_cpl(maia_move, board, stockfish) - get_cpl(moves[i], board, stockfish))
            except:
                continue

        else:
            try:
                maia_move = get_move(board, maia_b, "maia")
                cpl_b.append(-(get_cpl(maia_move, board, stockfish) - get_cpl(moves[i], board, stockfish)))
            except:
                continue
        board.push_san(moves[i])
    
    
    return (np.mean(cpl_w), np.mean(cpl_b))
    

def get_cpl(move, board, stockfish):
    board.push_san(move)
    stockfish.set_fen_position(board.fen())
    eval = stockfish.get_evaluation()
    board.pop()

    if (eval["type"] == "cp"):
        return eval["value"]
    else:
        return None
    
    



In [23]:
# Reading in the new lichess data 5000 games
path = "..\data\Curated\lichess_2024.ndjson"
temp = []

count = 0
with open(path, 'r') as file:
    for line in file:
        temp.append(json.loads(line))
        if count == 50:
            break

In [70]:
stockfish=Stockfish("../stockfish/stockfish-windows-x86-64-avx2.exe")
stockfish.set_depth(12)#How deep the AI looks
out = maia_cpl(temp[0], stockfish)

out


<UciProtocol (pid=26068)>: stderr >>        _
<UciProtocol (pid=26068)>: stderr >> |   _ | |
<UciProtocol (pid=26068)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=26068)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=7324)>: stderr >>        _
<UciProtocol (pid=7324)>: stderr >> |   _ | |
<UciProtocol (pid=7324)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=7324)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=26068)>: stderr >> Loading weights file from: lc0_windows\models\maia-1300.pb.gz
<UciProtocol (pid=26068)>: stderr >> Creating backend [cuda-auto]...


--weights=lc0_windows\models\maia-1300.pb.gz
--weights=lc0_windows\models\maia-1400.pb.gz


<UciProtocol (pid=26068)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=26068)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=26068)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=26068)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=26068)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=26068)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=26068)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=26068)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=7324)>: stderr >> Loading weights file from: lc0_windows\models\maia-1400.pb.gz
<UciProtocol (pid=7324)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=7324)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=7324)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=7324)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=7324)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<Uci

(-4.2, 1.6)

In [19]:
# Reading in the new lichess data 5000 games
path = "..\data\Curated\lichess_2024.ndjson"
games = []

count = 0
with open(path, 'r') as file:
    for line in file:
        games.append(json.loads(line))

In [5]:
model = "--weights=lc0_windows\models\maia-1100.pb.gz"
maia_1100 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1100.ndjson', 'w') as f:
    for entry in maia_1100:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

--weights=lc0_windows\models\maia-1100.pb.gz


<UciProtocol (pid=18640)>: stderr >>        _
<UciProtocol (pid=18640)>: stderr >> |   _ | |
<UciProtocol (pid=18640)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=18640)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=18640)>: stderr >> Loading weights file from: lc0_windows\models\maia-1100.pb.gz
<UciProtocol (pid=18640)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=18640)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=18640)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=18640)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=18640)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=18640)>: stderr >> GPU memory: 7.99939 Gb
<UciProtocol (pid=18640)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=18640)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=18640)>: stderr >> L2 cache capaci

In [6]:
model = "--weights=lc0_windows\models\maia-1200.pb.gz"
maia_1200 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1200.ndjson', 'w') as f:
    for entry in maia_1200:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

--weights=lc0_windows\models\maia-1200.pb.gz


<UciProtocol (pid=3196)>: stderr >>        _
<UciProtocol (pid=3196)>: stderr >> |   _ | |
<UciProtocol (pid=3196)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=3196)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=3196)>: stderr >> Loading weights file from: lc0_windows\models\maia-1200.pb.gz
<UciProtocol (pid=3196)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=3196)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=3196)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=3196)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=3196)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=3196)>: stderr >> GPU memory: 7.99939 Gb
<UciProtocol (pid=3196)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=3196)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=3196)>: stderr >> L2 cache capacity: 4194304


In [7]:
model = "--weights=lc0_windows\models\maia-1300.pb.gz"
maia_1300 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1300.ndjson', 'w') as f:
    for entry in maia_1300:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

<UciProtocol (pid=14780)>: stderr >>        _
<UciProtocol (pid=14780)>: stderr >> |   _ | |
<UciProtocol (pid=14780)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=14780)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=14780)>: stderr >> Loading weights file from: lc0_windows\models\maia-1300.pb.gz
<UciProtocol (pid=14780)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=14780)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=14780)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=14780)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=14780)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=14780)>: stderr >> GPU memory: 7.99939 Gb
<UciProtocol (pid=14780)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=14780)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=14780)>: stderr >> L2 cache capaci

--weights=lc0_windows\models\maia-1300.pb.gz


In [8]:
model = "--weights=lc0_windows\models\maia-1400.pb.gz"
maia_1400 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1400.ndjson', 'w') as f:
    for entry in maia_1400:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

--weights=lc0_windows\models\maia-1400.pb.gz


<UciProtocol (pid=24048)>: stderr >>        _
<UciProtocol (pid=24048)>: stderr >> |   _ | |
<UciProtocol (pid=24048)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=24048)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=24048)>: stderr >> Loading weights file from: lc0_windows\models\maia-1400.pb.gz
<UciProtocol (pid=24048)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=24048)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=24048)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=24048)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=24048)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=24048)>: stderr >> GPU memory: 7.99939 Gb
<UciProtocol (pid=24048)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=24048)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=24048)>: stderr >> L2 cache capaci

In [9]:
model = "--weights=lc0_windows\models\maia-1500.pb.gz"
maia_1500 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1500.ndjson', 'w') as f:
    for entry in maia_1500:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

<UciProtocol (pid=2992)>: stderr >>        _
<UciProtocol (pid=2992)>: stderr >> |   _ | |
<UciProtocol (pid=2992)>: stderr >> |_ |_ |_| v0.30.0 built Jul 21 2023
<UciProtocol (pid=2992)>: stderr >> Found configuration file: c:\pc_coding\Group-13_Chess_Project\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=2992)>: stderr >> Loading weights file from: lc0_windows\models\maia-1500.pb.gz
<UciProtocol (pid=2992)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=2992)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=2992)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=2992)>: stderr >> Latest version of CUDA supported by the driver: 12.5.0
<UciProtocol (pid=2992)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=2992)>: stderr >> GPU memory: 7.99939 Gb
<UciProtocol (pid=2992)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=2992)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=2992)>: stderr >> L2 cache capacity: 4194304


--weights=lc0_windows\models\maia-1500.pb.gz


In [ ]:
model = "--weights=lc0_windows\models\maia-1600.pb.gz"
maia_1600 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1600.ndjson', 'w') as f:
    for entry in maia_1600:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

In [ ]:
model = "--weights=lc0_windows\models\maia-1700.pb.gz"
maia_1700 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1700.ndjson', 'w') as f:
    for entry in maia_1700:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

In [ ]:
model = "--weights=lc0_windows\models\maia-1800.pb.gz"
maia_1800 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1800.ndjson', 'w') as f:
    for entry in maia_1800:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

In [ ]:
model = "--weights=lc0_windows\models\maia-1900.pb.gz"
maia_1900 = execute_maia_mm_v2(model, games)
# Save to a newline-delimited JSON file
with open('../data/curated/maia-1900.ndjson', 'w') as f:
    for entry in maia_1900:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')

In [23]:
stockfish = execute_stockfish_mm(games, 14)
# Save to a newline-delimited JSON file
with open('../data/curated/stockfish.ndjson', 'w') as f:
    for entry in stockfish:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')